# NGC3100
---
Things to investigate:
* Ionised gas density: do the dense regions track with the CO data?
* 

Plots to make:
* BPT 
* WHAN
* WHAV*


In [1]:
%matplotlib widget

In [74]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist

from spaxelsleuth.loaddata.s7 import load_s7_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, plot_BPT_lines
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_labels, bpt_colours
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dscatter, plot2dhistcontours

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
component_markers = ["o", ">", "D"]
rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [55]:
# Options
fig_path = "/priv/meggs3/u5708159/S7/figs/NGC3100/"
savefigs = True
eline_SNR_min = 3       # Minimum S/N of emission lines to accept


In [4]:
# Load the S7 sample
df = load_s7_galaxies(eline_SNR_min=eline_SNR_min,
                      sigma_gas_SNR_cut=True,
                      stekin_cut=True,
                      vgrad_cut=False)


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [5]:
# Load the SAMI sample
df_sami = load_sami_galaxies(ncomponents="recom",
                             bin_type="default",
                             eline_SNR_min=5, 
                             vgrad_cut=False,
                             correct_extinction=False,
                             sigma_gas_SNR_cut=True)

In [6]:
gal = "NGC3100"
df_gal = df[df["catid"] == gal]

## BPT diagrams
---

In [56]:
# Plot each component individually
col_y = "log O3"
col_x_list = ["log N2", "log S2", "log O1"]
col_z_list = ["BPT (numeric)", "sigma_gas", "r/R_e"]

for col_z in col_z_list:
    fig, axs, cax = plot_empty_BPT_diagram(nrows=1, colorbar=True)
    for cc, col_x in enumerate(col_x_list):
        for ii in range(3):
            plot2dscatter(df=df_gal, 
                          col_x=f"{col_x} (component {ii})",
                          col_y=f"{col_y} (component {ii})",
                          col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_gal else col_z,
                          ax=axs[cc], marker=component_markers[ii], markersize=50,
                          ymax=2.0, alpha=1.0,
                          plot_colorbar=True if cc == 2 else False,
                          cax=cax)
        # Decorations
        axs[cc].grid()
        axs[cc].set_ylabel("") if cc > 0 else None
        
    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"whan_whav_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## WHAN/WHAS/WHAV* diagrams
---

In [61]:
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z_list = ["BPT (numeric)", "r/R_e"]

for col_z in col_z_list:
    # Plot LZIFU measurements
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(13.7, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.03, bbox.height])
    for cc, col_x in enumerate(["log N2", "sigma_gas - sigma_*", "v_gas - v_*"]):
        # Plot full SAMI sample
        plot2dhistcontours(df=df_sami, col_x=f"{col_x} (total)" if col_x == "log N2" else f"{col_x}",
                           col_y=f"{col_y} (total)" if col_x == "log N2" else f"{col_y}",
                           col_z="count", log_z=True,
                           alpha=0.5, cmap="gray_r", ax=axs[cc],
                           plot_colorbar=False)
        # Plot the S7 data
        for ii in range(3):
            plot2dscatter(df=df_gal,
                          col_x=f"{col_x} (component {ii})",
                          col_y=f"{col_y} (component {ii})",
                          col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})"  in df_gal else col_z,
                          marker=markers[ii], ax=axs[cc], cax=cax,
                          alpha=1.0,
                          plot_colorbar=True if ii == 1 else False)

    # Decorations
    [ax.grid() for ax in axs]
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]
    [ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs[:-1]]
    [ax.axvline(0, ls="--", color="k") for ax in axs[1:]]

    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"whan_whav_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


## 2D map plots
---

In [80]:
# Stellar properties & others only defined for a single component
col_z_list = ["v_*", "sigma_*", "D4000", "A_V (total)"]

for col_z, fname in zip(col_z_list, fname_list):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4.6, 7))
    bbox = ax.get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.15, bbox.height])
    
    plot2dmap(df_gal=df_gal, bin_type="default", survey="s7", col_z=col_z, ax=ax,
              show_title=False, plot_colorbar=True, cax=cax,
              PA_deg=float(df_gal["WiFeS PA"].unique()[0]))
        
    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"2dmap_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


In [88]:
# Ionised gas properties
col_z_list = ["HALPHA", "HALPHA EW", "v_gas", "log sigma_gas", "sigma_gas - sigma_*", "BPT (numeric)", "S2 ratio"]
vmax_ha = np.nanmax(df_gal["HALPHA (component 0)"])  #

for col_z, fname in zip(col_z_list, fname_list):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(13.7, 7))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.05, bbox.height])
    
    for ii in range(3):
        plot2dmap(df_gal=df_gal, bin_type="default", survey="s7", col_z=f"{col_z} (component {ii})", ax=axs[ii],
                  show_title=False, plot_colorbar=True if ii == 2 else False, cax=cax,
                  vmax=vmax_ha if col_z == f"HALPHA" else None,
                  PA_deg=float(df_gal["WiFeS PA"].unique()[0]))
        
    # Decorations
    axs = fig.get_axes()
    for ax in axs[2:]:
        ra = ax.coords[0]
        dec = ax.coords[1]
        dec.set_ticks_visible(False)
        dec.set_ticklabel_visible(False)
        
    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"2dmap_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


## Extinction
---

In [87]:
# Halpha/Hbeta ratio
for ii in range(3):
    df_gal[f"HaHbratio (component {ii})"] = df_gal[f"HALPHA (component {ii})"] / df_gal[f"HBETA (component {ii})"]
col_z_list = ["HaHbratio"]
vmax = 2.86 + 1
vmin = 2.86 - 1

for col_z, fname in zip(col_z_list, fname_list):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(13.7, 7))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, 0.05, bbox.height])
    
    for ii in range(3):
        plot2dmap(df_gal=df_gal, bin_type="default", survey="s7", col_z=f"{col_z} (component {ii})", ax=axs[ii],
                  show_title=False, plot_colorbar=True if ii == 2 else False, cax=cax,
                  vmax=vmax, vmin=vmin,
                  PA_deg=float(df_gal["WiFeS PA"].unique()[0]))
        
    # Decorations
    axs = fig.get_axes()
    for ax in axs[2:]:
        ra = ax.coords[0]
        dec = ax.coords[1]
        dec.set_ticks_visible(False)
        dec.set_ticklabel_visible(False)
        
    # Save 
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"2dmap_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal.loc[:, "x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))


/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:128: RuntimeWarning: invalid value encountered in log10
  ax.contour(np.log10(im_B) + 15, linewidths=linewidths, colors=colors, levels=10 if levels is None else levels)


In [79]:
df_gal.columns


Index(['A_V (total)', 'A_V error (total)', 'BPT (component 2)',
       'BPT (component 0)', 'BPT (component 1)', 'BPT (global)',
       'BPT (numeric) (component 2)', 'BPT (numeric) (component 0)',
       'BPT (numeric) (component 1)', 'BPT (numeric) (total)',
       ...
       'v_gas error (component 2)', 'v_gas error (component 0)',
       'v_gas error (component 1)', 'v_gas error (total)',
       'x (projected, arcsec)', 'x0 (pixels)', 'y (projected, arcsec)',
       'y0 (pixels)', 'z_spec', 'x, y (pixels)'],
      dtype='object', length=412)